# Incursion dans la programmation fonctionnelle : récursivité
![Fonctionnel](http://modelagnostic.co.uk/images/a/3/9/9/4/a39945ca092081f9b7dbf18932bc1f3ec93a184e-screen-shot-2018-11-12-at-103020.png)
## 1.Le style de programmation fonctionnel
Pour calculer la somme d'un tableau de nombres...On a pas le choix, il va falloir additionner chacune des valeurs contenues dans le tableau. On peut proposer quelque chose comme ça :

In [1]:
# Version ABJECTE, ne pas faire ça à la maison...

# Variables globales
liste = [1,2,3]

# Fonctions

def sommeIteratif():
    somme = 0
    for i in range(len(liste)):
        somme = somme + liste[0]
        liste.remove(liste[0])       
    print(somme)

    
# Programme principal 

sommeIteratif()

Ce programme, "fonctionne" (ça se discute, qu'est-ce qu'il est sensé faire au juste ?) mais il a de nombreuses faiblesses : 
* La fonction ```sommeIteratif``` utilise des variables du programme principal directement, elle ne peut donc pas être réexploitée dans un nouveau contexte et ne peut même pas être testée en dehors de ce programme précis !
* La fonction traite elle même l'affichage, si je veux utiliser cette fonction pour calculer une moyenne, je ne peux pas
* Elle ne peut pas être exécutée plusieurs fois, deux exécutions successives ne donnent pas le même résultat (elle modifie une variable extérireure) on dit qu'elle a un **effet de bord** ou encore qu'elle n'est pas **réentrante**

---
### RETENIR

Pour éviter ces problèmes on peut adopter un "style" fonctionnel, en se conformant aux contraintes suivantes : 
* les traitements devraient être contenus dans des fonctions
* les fonctions utilisent exclusivement leurs paramètres, 
* elles ne modifient aucune variable extérieure
* elles ne doivent avoir aucun effet de bord (affichage ou saisie par exemple...)
---

In [2]:
# Version impérative sans effet de bord

# Pas de variables globales

# Fonctions

def sommeIteratif(liste):
    somme = 0
    for i in range(len(liste)):
        somme = somme + liste[i]
    return somme

# Programme principal

print(sommeIteratif([1,2,3]))

# la fonction est testable
assert(sommeIteratif([1,2,3]) == 6)
# et réentrante
assert(sommeIteratif([1,2,3]) == 6)

# et qui permet d'être utilisée pour d'autres fonctions
def moyenneIteratif(liste):
    return sommeIteratif(liste) / len(liste)

print(moyenneIteratif([1,2,3]))

6
2.0


Si on veut aller plus loin, on peut chercher à se débarasser de tout ce qui n'est pas à proprement parler fonctionnel...
On pourrait par exemple chercher à se débarasser de la boucle...

Idée : Pour parcourir un tableau, on commence par regarder le premier élément, puis...on parcourt le tableau constitué des éléments qui reste (qui est strictement plus court). Si le tableau que l'on souhaite parcourir est vide, on ne fait rien.


In [3]:
def sommeFonctionelle(liste):
    # si la liste est vide
    if (len(liste) == 0):
        return 0
    # sinon on extrait la première valeur, et on fait la somme de ce qui reste
    return liste[0] + sommeFonctionelle(liste[1:])

# testable
print(sommeFonctionelle([1,2,3]))
assert(sommeFonctionelle([1,2,3]) == 6)

6


Cette nouvelle proposition est "encore plus" fonctionnelle (elle n'utilise pas de boucles) mais pour arriver à ce résultat, elle exploite un mécanisme particulièrement étrange : **la fonction sommeFonctionnelle s'appelle elle même !**

On dit que cette fonction est **récursive**.

---
### RETENIR

On dit d'une fonction qui contient un appel recursif (qui s'appelle elle-même) que c'est une _fonction récursive_

---

## 2.Programmation récursive

Cette vision est à rapprocher des suites définies par récurrence. 

Pour calculer $S_n=1 + ... + n$ on peut donner une définition itérative de cette suite : $$S_n = \sum_{i=0}^{i=n} n$$
On calcule alors $$S_1 = 1$$ $$S_2 = 1 + 2 = 3$$ $$S_3 = 1+2+3=6$$ $$S_4 = 1 + 2 + 3 + 4 = 10 $$  

Mais on peut aussi choisir de définir cette suite par récurrence par  :
$$\begin{cases}
S_0 = 0 \\
S_n = n + S_{n-1}
\end{cases}
$$
Autrement dit, la somme des $n$ premiers entiers est égale à la somme des $n-1$ premiers entiers à laquelle on ajoute $n$.

En utilisant cette définition, on calcule alors 
$$ S_0 = 0$$
$$ S_1 = 1 + S_0 = 1 + 0 = 1$$
$$ S_2 = 2 + S_1 = 2 + 1 = 3$$
$$ S_3 = 3 + S_2 = 3 + 3 = 6$$
$$ S_4 = 4 + S_3 = 4 + 6 = 10$$


### 2.1 Etude de la suite des appels de la somme "fonctionnelle"
Modifions un peu la fonction de départ pour que celle-ci indique ce qui se passe : 
* on ajoute une trace (qu'il faudra enlever, c'est un effet de bord indésirable !) à l'entrée de la fonction
* on modifie un peu le retour pour afficher la valeur de retour avant que celle-ci soit exécutée

In [4]:
def sommeFonctionelle(liste):
    print("  "*len(liste), "Appel sommeFonctionnelle avec ", liste, " de taille ", len(liste))
    # si la liste est vide
    if (len(liste) == 0):
        print(" Liste vide, je retourne 0")
        return 0
    # sinon on extrait la première valeur, et on fait la somme de ce qui reste
    valeurRetour = liste[0] + sommeFonctionelle(liste[1:])
    print("  "*(len(liste)), "Retour de sommeFonctionnelle avec ", liste, " de taille ", len(liste), " --> ", valeurRetour)
    return valeurRetour 

# testable

print(sommeFonctionelle([1,2,3]))

       Appel sommeFonctionnelle avec  [1, 2, 3]  de taille  3
     Appel sommeFonctionnelle avec  [2, 3]  de taille  2
   Appel sommeFonctionnelle avec  [3]  de taille  1
 Appel sommeFonctionnelle avec  []  de taille  0
 Liste vide, je retourne 0
   Retour de sommeFonctionnelle avec  [3]  de taille  1  -->  3
     Retour de sommeFonctionnelle avec  [2, 3]  de taille  2  -->  5
       Retour de sommeFonctionnelle avec  [1, 2, 3]  de taille  3  -->  6
6


---

### RETENIR


``` 
sommeFonctionelle([1,2,3]) = return 1 + sommeFonctionnelle([2,3])
                                                |
                                                return 2 + sommeFonctionnelle([3])
                                                                    |
                                                                    return 3 + sommeFonctionnelle([])
                                                                                        |
                                                                                        0


```
Cette manière de présenter l'exécution d'un programme en indiquant les appels effectués est appelée un **arbre d'appel**.  

---

### 2.2 Comment construire une fonction récursive  
---

### RETENIR

Une fonction récursive s'appelle elle-même. 
Le point le plus fondamental auquel il faut faire attention est...que la récursion doit s'arrêter.
* On définit donc les conditions d'arrêt de la fonction (qui vont déclencher un ```return``` qui ne fera pas d'appel récursif) : on parle des **cas de base**
* La définition de la partie récursive est souvent plus simple, mais il faut faire attention à ne pas construire d'appels avec des paramètres invalides. 

---



## 3. Exercices classiques 

### 3.1 Classique : la factorielle. 
![Factorielle](http://villemin.gerard.free.fr/Wwwgvmm/Compter/FactProp_fichiers/image013.jpg)
La factorielle $n!$ est définie comme le produit des $n$ premiers entiers. 
On a donc $$n! = 1 \times 2 \times \ldots n$$

Ecrire deux implémentations de la fonction factorielle, une itérative, l'autre récursive.

In [5]:
def factIteratif(n):
    """Calcul de n! (version itérative)

    Args:
        n: le nombre dont on calcule la factorielle

    Returns:
        n! si n > 1, 0 sinon

    """
    if (n<=0):
        return 0
    prod = 1
    for i in range(n):
        prod = prod * (i+1)
    return prod

def factRecursif(n):
    """Calcul de n! (version récursive)

    Args:
        n: le nombre dont on calcule la factorielle

    Returns:
        n! si n > 1, 0 sinon

    """
    if (n <=0):
        return 0
    if (n==1):
        return 1
    return n * factRecursif(n-1)


In [6]:
# NE PAS MODIFIER MAIS EXECUTER APRES AVOIR REALISE l'EXERCICE
# JEUX DE TESTS
assert(factIteratif(-10) == 0)
assert(factIteratif(1) == 1)
assert(factIteratif(2) == 2)
assert(factIteratif(3) == 6)
assert(factIteratif(10) == 3628800)
for i in range(-10, 10):
    assert(factIteratif(i) == factRecursif(i))

### 3.2 Classique : la suite de syracuse
![Syracuse](http://villemin.gerard.free.fr/Wwwgvmm/Iteration/Syracus2_fichiers/image019.jpg)
En mathématiques, on appelle suite de Syracuse une suite d'entiers naturels définie de la manière suivante : 
* on part d'un nombre entier plus grand que zéro 
* s’il est pair, on le divise par 2 
* s’il est impair, on le multiplie par 3 et on ajoute 1. 
En répétant l’opération, on obtient une suite d'entiers positifs dont chacun ne dépend que de son prédécesseur.


Par exemple, à partir de 14, on construit la suite des nombres : 14, 7, 22, 11, 34, 17, 52, 26, 13, 40, 20, 10, 5, 16, 8, 4, 2, 1, 4, 2… C'est ce qu'on appelle la suite de Syracuse du nombre 14.


Après que le nombre 1 a été atteint, la suite des valeurs (1,4,2,1,4,2…) se répète indéfiniment en un cycle de longueur 3, appelé cycle trivial.


Si l'on était parti d'un autre entier, en lui appliquant les mêmes règles, on aurait obtenu une suite de nombres différente. A priori, il serait possible que la suite de Syracuse de certaines valeurs de départ n'atteigne jamais la valeur 1, soit qu'elle aboutisse à un cycle différent du cycle trivial, soit qu'elle diverge vers l'infini. Or, on n'a jamais trouvé d'exemple de suite obtenue suivant les règles données qui n'aboutisse pas à 1, puis au cycle trivial.

La conjecture de Syracuse, encore appelée conjecture de Collatz, conjecture d'Ulam, conjecture tchèque ou problème 3x + 1, est l'hypothèse mathématique selon laquelle la suite de Syracuse de n'importe quel entier strictement positif atteint 1.

En dépit de la simplicité de son énoncé, cette conjecture défie depuis de nombreuses années les mathématiciens. Paul Erdős a dit à propos de la conjecture de Syracuse : "les mathématiques ne sont pas encore prêtes pour de tels problèmes".

* a) Ecrire une fonction récursive qui calcule la valeur suivant $n$ de la suite de Syracuse et faire afficher (non récursif) la suite de Syracuse pour $n=15$ tant que celle-ci ne prend pas la valeur $1$.

* BONUS :
    * b) On appelle "temps de vol" d'un nombre $n$ le nombre d'étapes pour que la suite de Syracuse atteigne la valeur $1$.
    
    Vérifier que le "temps de vol" de $77671$ est $231$
    
    * c) Le plus grand nombre obtenu dans la suite est appelé "l'altitude maximale du vol", vérifier que l'altitude maximale de vol pour $77671$  est $1570824736$


In [7]:
def syracuse(n):
    if (n % 2 == 0):
        return n/2
    return 3*n + 1

n = 77671
altitude = n
tempsVol = 0
while (n>1):
    # print("N=", n)
    if (altitude < n):
        altitude = n
    tempsVol = tempsVol + 1
    n = syracuse(n)
print("Terminé, n=",n, " altitude=", altitude, " temps de vol = ", tempsVol)
    

Terminé, n= 1.0  altitude= 1570824736.0  temps de vol =  231


In [8]:
# NE PAS MODIFIER MAIS EXECUTER APRES AVOIR REALISE l'EXERCICE
# JEUX DE TESTS
assert(factIteratif(-10) == 0)
assert(factIteratif(1) == 1)

### 3.3 Classique : Les lapins de Fibonacci
![Lapins](http://www.maths.surrey.ac.uk/hosted-sites/R.Knott/Fibonacci/manyrab.gif)
